In [9]:
%pip install pygame -quiet

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\User\Documents\PDF and sheit\BME Solar Boat\SolarBoatFrame\.venv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\User\Documents\PDF and sheit\BME Solar Boat\SolarBoatFrame\.venv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\User\Documents\PDF and sheit\BME Solar Boat\SolarBoatFrame\.venv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\User\Documents\PDF and sheit\BME Solar Boat\SolarBoatFrame\.venv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\User\Documents\PDF and sheit\BME Solar Boat\SolarBoatFrame\.venv\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


In [10]:
import numpy as np
from gps_coordinate.base import GPSPoint

from loguru import logger
logger.remove()

In [11]:
# LK_COEFFS = {
#     0: -0.02348713,
#     1: 2.10745127,
#     2: 0.01235089,
#     3: -0.004935727,
#     4: 1.430259e-04,
#     5: -1.561371e-06,
#     6: 5.987527e-09
# }

# print([f"{key}: {item:.5f}" for key, item in LK_COEFFS.items()])

# def compute_Lk(delta_psi_deg: float) -> float:
#     """Compute the lookahead distance Lk for turn initiation, given Δψ in degrees"""
#     return sum(delta_psi_deg**p * LK_COEFFS[p] for p in range(7))  # p=0..6

In [12]:
# Rectangle corners (latitude, longitude) - roughly 30x20 meter box

W, E = 17.642, 17.643
S, N = 46.782, 46.783

boat_origin = GPSPoint(S - 0.002, W)
boat = GPSPoint(boat_origin.latitude, boat_origin.longitude)

corner_1 = GPSPoint(N, W)
corner_2 = GPSPoint(N, E)
corner_3 = GPSPoint(S, E)
corner_4 = GPSPoint(S, W)


# Ordered loop of waypoints
waypoints = [corner_1, corner_2]
boat, waypoints

(GPSPoint(lat=46.78000, lon=17.64200),
 [GPSPoint(lat=46.78300, lon=17.64200), GPSPoint(lat=46.78300, lon=17.64300)])

In [13]:
# Constants
U = 1.1  # [m/s]
DT = 0.2  # [s]
MAX_STEPS = 1000
RUDDER_LIMIT_DEG = 35
ARRIVAL_TOL_M = 10.0

# Controller gains (from paper)
k1, k2, k3, k4 = 1.6, 19.92, 2.125, 92.1

In [14]:
from enum import Enum, auto
import numpy as np
from gps_coordinate.base import GPSPoint
from loguru import logger

# Constants
DT = 0.2  # timestep
ARRIVAL_TOL_M = 10.0
RUDDER_LIMIT_DEG = 35
MIN_SPEED = 0.1
CRUISE_SPEED = 2
SPEED_STEP = 0.01

# Controller gains (Tomera PDPI)
k1, k2, k3, k4 = 1.6, 19.92, 2.125, 92.1


class Mode(Enum):
    TRACKING = auto()
    TURNING = auto()


def simulate_path_follower_fsm(boat: GPSPoint, waypoints: list[GPSPoint]):
    if not waypoints:
        return

    mode = Mode.TRACKING
    int_ey = 0
    error_psi_prev = 0
    psi = np.pi / 2  # heading north

    U = 0.0

    current_wp = waypoints.pop(0)

    while True:
        dx = current_wp.Xn - boat.Xn
        dy = current_wp.Yn - boat.Yn
        dist = np.hypot(dx, dy)

        if dist < ARRIVAL_TOL_M:
            if not waypoints:
                logger.info("Reached final waypoint.")
                return
            current_wp = waypoints.pop(0)
            mode = Mode.TURNING
            int_ey = 0
            continue

        # Desired heading
        psi_d = np.arctan2(current_wp.Yn - boat.Yn, current_wp.Xn - boat.Xn)
        error_psi = np.arctan2(np.sin(psi - psi_d), np.cos(psi - psi_d))
        de_psi = (error_psi - error_psi_prev) / DT
        error_psi_prev = error_psi

        # Lateral error
        ey = np.sin(psi_d) * (boat.Xn - current_wp.Xn) - np.cos(psi_d) * (boat.Yn - current_wp.Yn)

        if mode == Mode.TURNING:
            U = max(U - SPEED_STEP, MIN_SPEED)
            delta = k1 * error_psi + k2 * de_psi
            rudder = np.clip(delta, -RUDDER_LIMIT_DEG, RUDDER_LIMIT_DEG)

            # Exit turning mode if aligned
            if np.abs(error_psi) < np.deg2rad(5) and np.abs(de_psi) < np.deg2rad(0.5):
                mode = Mode.TRACKING
                int_ey = 0

        elif mode == Mode.TRACKING:
            U = min(U + SPEED_STEP, CRUISE_SPEED)
            int_ey += ey * DT
            delta_PD = k1 * error_psi + k2 * de_psi
            delta_PI = k3 * ey + k4 * int_ey
            rudder = np.clip(delta_PD + delta_PI, -RUDDER_LIMIT_DEG, RUDDER_LIMIT_DEG)

        # Apply rudder effect
        rudder_rad = np.deg2rad(rudder)
        psi += -rudder_rad * 0.05  # simple model

        dx_local = U * np.cos(psi + rudder_rad) * DT
        dy_local = U * np.sin(psi + rudder_rad) * DT

        new_x = boat.Xn + dx_local
        new_y = boat.Yn + dy_local
        boat.set_from_Xn_Yn(new_x, new_y)

        yield new_x, new_y, current_wp.Xn, current_wp.Yn, dy_local, dx_local, mode.name, dist, U

In [15]:
# Pygame setup
import pygame

pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Boat Path Simulation")
clock = pygame.time.Clock()


def world_to_screen(x, y, origin_x, origin_y, scale):
    return int((x - origin_x) * scale + WIDTH / 2), int(HEIGHT / 2 - (y - origin_y) * scale)

In [16]:
# Get bounds for scaling
all_x = [wp.Xn for wp in waypoints]
all_y = [wp.Yn for wp in waypoints]

padding = 50  # pixels
max_span = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
scale = (min(WIDTH, HEIGHT) - 2 * padding) / max_span

origin_x = (max(all_x) + min(all_x)) / 2
origin_y = (max(all_y) + min(all_y)) / 2

scale = 1.5  # screen pixels per meter

In [17]:
# Generator
path_gen = simulate_path_follower_fsm(boat, waypoints)

In [18]:
# Auto scale based on bounding box

font = pygame.font.SysFont(None, 24)

In [19]:
# Simulation loop
running = True
while running:
    screen.fill((40, 130, 255))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    try:
        bx, by, wx, wy, dy_local, dx_local, wp_idx, dist, U = next(path_gen)
    except StopIteration:
        running = False
        continue

    # Draw waypoints
    for idx, wp in enumerate(waypoints):
        x, y = world_to_screen(wp.Xn, wp.Yn, origin_x, origin_y, scale)
        pygame.draw.circle(screen, (0, 0, 255), (x, y), 5)
        wp_text = font.render(f'{idx}', True, (0, 0, 0))
        screen.blit(wp_text, (x + 5, y + 5))

    # Draw current waypoint target
    wxs, wys = world_to_screen(wx, wy, origin_x, origin_y, scale)
    pygame.draw.circle(screen, (0, 255, 0), (wxs, wys), 6)

    # Draw boat
    bx_s, by_s = world_to_screen(bx, by, origin_x, origin_y, scale)
    pygame.draw.circle(screen, (255, 0, 0), (bx_s, by_s), 6)

    # Heading arrow
    heading_angle = np.arctan2(dy_local, dx_local)
    arrow_len = 20
    hx = bx_s + arrow_len * np.cos(heading_angle)
    hy = by_s - arrow_len * np.sin(heading_angle)
    pygame.draw.line(screen, (255, 0, 0), (bx_s, by_s), (hx, hy), 2)

    # Line to next waypoint
    pygame.draw.line(screen, (0, 150, 0), (bx_s, by_s), (wxs, wys), 2)

    # Display waypoint info
    info = font.render(f"WP[{wp_idx}]  Dist: {dist:.1f} m | SPeed is {U} m/s", True, (0, 0, 0))
    screen.blit(info, (10, 10))

    pygame.display.flip()
    clock.tick(240)

In [20]:
pygame.quit()